<a href="https://colab.research.google.com/github/aithaprasad/NLP_Celtic_Mutation/blob/main/Celtic_Mutation_Scratch_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Reading the data from the respective files.

In [1]:
import csv
with open('train.tsv', encoding="utf-8") as file:
  f = csv.reader(file, delimiter="\t")
  all_data = []
  for line in f:
    all_data.append(line)

In [2]:
len(all_data)

5057059

##### Function to just split the data. Re-used from the first competition

In [3]:
def train_test_split(data, percent = 0.1):
    percent = int(percent * 10)

    train = data[((len(data) // 10) * percent):]
    test = data[:((len(data) // 10) * percent)]

    return train, test

In [4]:
train, test = train_test_split(all_data)

##### As we have five tags. The below code keeps the count of different tags on each unique word. I guess the names for the variables speak for what they actually are.

In [5]:
word_tag_count = dict()
N_count, S_count, U_count, T_count, H_count = 0, 0, 0, 0, 0 
unique_words = []
for word_tag in train:
  word, tag = word_tag[0], word_tag[1]
  
  if word == "<S>": continue
  
  if tag == 'N': N_count += 1
  elif tag == 'S': S_count += 1
  elif tag == 'T': T_count += 1
  elif tag == 'U': U_count += 1
  else: H_count += 1

  if word not in word_tag_count:
    word_tag_count[word] = {tag : 1}
    unique_words.append(word)
  else:
    if tag not in word_tag_count[word]: word_tag_count[word][tag] = 1
    else: word_tag_count[word][tag] += 1

In [6]:
print(len(word_tag_count))
print(len(unique_words))
print(N_count)
print(S_count)
print(T_count)
print(U_count)
print(H_count)

118690
118690
3725660
443398
16030
149449
36456


##### We are manipulating the above "word_tag_count" dictionary with the probabilities(each word's tag has the probability of that tag for that word). For example, the word "tug". it is a 'S' with 0.005895380673796454 probability.

In [7]:
total_count_per_tag = {'N' : N_count, 'S': S_count, 'T' : T_count, 'U': U_count, 'H' : H_count}
for word in unique_words:
  new_dict = {'N' : 0, 'S': 0, 'T' : 0, 'U': 0, 'H' : 0}
  tags = ['N', 'S', 'U', 'T', 'H']
  for tag in tags:
    if tag in word_tag_count[word]:
      new_dict[tag] = word_tag_count[word][tag] / total_count_per_tag[tag] 
  word_tag_count[word] = new_dict

In [8]:
total_count_per_tag

{'N': 3725660, 'S': 443398, 'T': 16030, 'U': 149449, 'H': 36456}

In [9]:
word_tag_count['tug']

{'N': 2.1472705507212145e-06,
 'S': 0.005895380673796454,
 'T': 0,
 'U': 0.0002944148170947949,
 'H': 0}

##### We are constructing the dictionary which stores the context for each sentence like how many times a 'N' has followed 'N' or basically, how many times what tag is followed by what tag. 

In [10]:
sentence_context_tag_count = {'<S>' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'N' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 
                              'S' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'T' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0},
                              'U' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'H' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}}

for i in range(1, len(train)):
  if train[i][0] == '<S>': continue
  word, tag = train[i][0], train[i][1]
  prev_word, prev_tag = train[i - 1][0], train[i - 1][1]
  if prev_word == '<S>': prev_tag = '<S>'
  sentence_context_tag_count[prev_tag][tag] += 1

In [11]:
sentence_context_tag_count

{'<S>': {'N': 162620, 'S': 17221, 'T': 14, 'U': 315, 'H': 190},
 'N': {'N': 2953011, 'S': 395229, 'T': 15969, 'U': 148486, 'H': 35338},
 'S': {'N': 417378, 'S': 22911, 'T': 28, 'U': 516, 'H': 692},
 'T': {'N': 14970, 'S': 946, 'T': 3, 'U': 2, 'H': 41},
 'U': {'N': 142934, 'S': 5702, 'T': 12, 'U': 100, 'H': 107},
 'H': {'N': 34746, 'S': 1389, 'T': 4, 'U': 30, 'H': 88}}

##### We utilize the count and convert them to probabilities with 1-smoothing.

In [12]:
for tag in ['N', 'S', 'T', 'U', 'H', '<S>']:
  total_tag_sum = sum(sentence_context_tag_count[tag].values())
  for next_tag in ['N', 'S', 'T', 'U', 'H']:
    sentence_context_tag_count[tag][next_tag] = (1 + sentence_context_tag_count[tag][next_tag]) / total_tag_sum

In [13]:
sentence_context_tag_count

{'<S>': {'N': 0.9016467065868263,
  'S': 0.0954868041694389,
  'T': 8.316699933466401e-05,
  'U': 0.001752051452650255,
  'H': 0.0010589931248613884},
 'N': {'N': 0.8322955282546696,
  'S': 0.111394116120115,
  'T': 0.004501085531053404,
  'U': 0.04185051266434106,
  'H': 0.009960166661358561},
 'S': {'N': 0.9453122699733877,
  'S': 0.05189287129834098,
  'T': 6.568144499178982e-05,
  'U': 0.0011709416227846667,
  'H': 0.0015695600475624256},
 'T': {'N': 0.937915048239569,
  'S': 0.05932840496178424,
  'T': 0.0002505951635133442,
  'U': 0.00018794637263500814,
  'H': 0.002631249216890114},
 'U': {'N': 0.9602297537872426,
  'S': 0.038312451714755974,
  'T': 8.733331094017668e-05,
  'U': 0.0006785126465352188,
  'H': 0.0007255382755030063},
 'H': {'N': 0.9583528697906611,
  'S': 0.03833742449733844,
  'T': 0.00013790440466668506,
  'U': 0.0008550073089334474,
  'H': 0.002454698403066994}}

##### This is basically the predict function which is Viterbi, where we utlize the above probabilities and assign the probability for the tag and store the max state(probability till that point) and at the end we return the sentence and the respective predicted tags.

In [14]:
def viterbi(sentence):
  state = []
  tags = ['N', 'S', 'U', 'T', 'H']
  for key, word in enumerate(sentence):
    p = []
    for tag in tags:
      emission_p = 0
      if key == 0 or word == '<S>': transmission_prob = sentence_context_tag_count['<S>'][tag]
      else: transmission_prob = sentence_context_tag_count[state[-1]][tag]
      if word in word_tag_count.keys(): emission_p = word_tag_count[word][tag]
  
      state_prop = emission_p * transmission_prob
      p.append(state_prop)
    max_value=max(p)
    value=tags[p.index(max_value)]
    state.append(value)
  return list(zip(sentence, state)), state

In [15]:
test_data = []
test_label = []
for data in test:
  test_data.append(data[0])
  test_label.append(data[1])

In [16]:
predicted_pairs, pred_val = viterbi(test_data)

In [17]:
#correct, total = 0, len(test_label)
#for i in range(len(test_label)):
#  if test_label[i] == pred_val[i]: correct += 1
#print(correct / total)

In [27]:
def predict_from_scratch(sentence_tag):
  
  correct = 0
  sentence, tags = [], []
  
  for word, tag in sentence_tag:
    sentence.append(word)
    tags.append(tag)
  
  pred_labels = viterbi(sentence)
  for i in range(len(tags)):
    if pred_labels[1][i] == tags[i]:
      correct += 1
  
  return correct


In [47]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  #correct_anything_goes = 0
  #testfile = open('test.tsv', 'r')
  #testfile = open('drive/MyDrive/NLP_Assignments/celtic_train.tsv', 'r')
  testfile = test#[0:505705]
  sentence = []
  for line in testfile:
    total += 1
    pieces = line #line.rstrip("\n").split("\t") 
    if pieces[0]=='<S>':
      correct_from_scratch += predict_from_scratch(sentence)
      #correct_anything_goes += predict_anything_goes(sentence)
      sentence = []
    else:
      sentence.append(pieces)
  correct_from_scratch += predict_from_scratch(sentence)
  #correct_anything_goes += predict_anything_goes(sentence)
  return correct_from_scratch/total#(, correct_anything_goes/total)

In [48]:
evaluate()

0.8695227454741401

##References:

https://www.mygreatlearning.com/blog/pos-tagging/ 

I used this for some algo understanding purposes, but did not take any code from it.